In [ ]:
# Mapping both CALC and BLS data and doing visual comparisons of DC versus non-DC roles

In [ ]:
import pandas as pd
from uszipcode import SearchEngine
import math

In [ ]:
# read in CALC data
calc = pd.read_csv("/Users/Jeanine/Downloads/CALC_predictive_modeling_withlocation1.csv", sep = "|")

In [ ]:
# zipcode lookup
search = SearchEngine(simple_zipcode=True)

In [ ]:
# create lists for the zipcode information
lat_list = []
lng_list = []
city_list = []
state_list = []

In [ ]:
# get lat, long, city, and state for each row
z = 0
while z < len(calc):
    try:
        zip_return = search.by_zipcode(str(math.trunc(calc['zipcode'][z])))
    except ValueError:
        z += 1
        lat_list.append("")
        lng_list.append("")
        city_list.append("")
        state_list.append("")
    lat_list.append(zip_return.lat)
    lng_list.append(zip_return.lng)
    city_list.append(zip_return.city)
    state_list.append(zip_return.state)
    z += 1

In [ ]:
# apply the data to the CALC dataset
calc['latitude'] = lat_list
calc['longitude'] = lng_list
calc['city'] = city_list
calc['state'] = state_list

In [ ]:
# replace empty values with 0s for processing
calc['latitude'].replace(['', 'None'], 0, inplace=True)
calc['latitude'] = calc['latitude'].fillna(0)
calc['longitude'].replace(['', 'None'], 0, inplace=True)
calc['longitude'] = calc['longitude'].fillna(0)
calc.head(50)

In [ ]:
# create list of locations that match bls clustering

In [ ]:
# read in csv with BLS areas and their zipcode
bls_areas = pd.read_csv("/Users/Jeanine/Documents/DAEN690/bls_areas.csv")

In [ ]:
# get lat and long for BLS areas
lat_list = []
lng_list = []

z = 0
while z < len(bls_areas):
    try:
        zip_return = search.by_zipcode(str(bls_areas['zipcode'][z]))
    except ValueError:
        z += 1
        lat_list.append("")
        lng_list.append("")
    lat_list.append(zip_return.lat)
    lng_list.append(zip_return.lng)
    z += 1

bls_areas['latitude'] = lat_list
bls_areas['longitude'] = lng_list

In [ ]:
bls_areas

In [ ]:
# calculate the distance between two locations
from math import radians, cos, sin, asin, sqrt
def dist(lat1, long1, lat2, long2):
    # convert decimal degrees to radians 
    lat1, long1, lat2, long2 = map(radians, [lat1, long1, lat2, long2])
    # haversine formula 
    dlon = long2 - long1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # radius of earth in kilometers is 6371
    km = 6371* c
    return km

In [ ]:
# determine shortest distance
def find_nearest(lat, long):
    distances = bls_areas.apply(
        lambda row: dist(lat, long, row['latitude'], row['longitude']), 
        axis=1)
    return bls_areas.loc[distances.idxmin(), 'region']

In [ ]:
# add nearest BLS location to each CALC row
calc['nearest_loc'] = calc.apply(
    lambda row: find_nearest(row['latitude'], row['longitude']), 
    axis=1)

In [ ]:
# export CALC location date
calc.to_csv("/Users/Jeanine/Documents/DAEN690/calc_scrape_w_locbins.csv", sep = "|")

In [ ]:
# create df with the occurrences of each state in CALC dataset
from collections import Counter
state_ct = Counter(calc['state'])
dfstate = pd.DataFrame.from_dict(state_ct, orient='index').reset_index()

In [ ]:
# rename the columns
dfstate = dfstate.rename(columns={"index": "state", 0: "count"})

In [ ]:
# get the FIPS value for each state
df_sample = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/laucnty16.csv')

In [ ]:
# get only the state code from FIPS dataset
dfsampsplit = []
s = 0
while s < len(df_sample):
    dfsampsplit.append(df_sample['County Name/State Abbreviation'][s].split(", ")[-1])
    s+=1
df_sample['state'] = dfsampsplit
dic = {'District of Columbia': 'DC'}
df_sample = df_sample.replace(dic)

In [ ]:
# create a df of just the state codes and FIPS based on above result
(df_sample['state'].append(df_sample['State FIPS Code'])).unique()
data = {'state':['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA',
       'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA',
       'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY',
       'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
       'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY', 'PR'],
        'fips':[1, 2, 4, 5, 6, 8,
       9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26,
       27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 44,
       45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 56, 72]}
statefips = pd.DataFrame(data)

In [ ]:
# merge with the CALC counts
dfstate = dfstate.merge(statefips, how='left', on='state')

In [ ]:
# remove blanks and NA counts
dfstate = dfstate.drop(dfstate.index[[4, 18]])

In [ ]:
dfstate['count'] = pd.to_numeric(dfstate['count'])

In [ ]:
# plot state counts from CALC data
import plotly.express as px

fig = px.choropleth(dfstate, locations="state", color="count", 
                    locationmode="USA-states",
                    scope="usa",
                    color_continuous_scale=px.colors.sequential.Sunsetdark,
                    range_color=(0,dfstate['count'].max()),
                    title="GSA CALC Contracts by State")
fig.update_layout(coloraxis_colorbar=dict(
    title="Total Number of Contracts",
    thicknessmode="pixels", thickness=50,
    lenmode="pixels", len=350,
    yanchor="top", y=1,
    ticks="outside"
))
fig.show()

In [ ]:
# determine if job is in the DC region
is_dc_bin = []

x = 0
while x < len(calc):
    if calc['nearest_loc'][x] == 'Washington-Arlington-Alexandria, DC-VA-MD-WV Metropolitan Division':
        is_dc_bin.append(1)
    else:
        is_dc_bin.append(0)
    x += 1
    
calc['is_dc_bin'] = is_dc_bin 
    

In [ ]:
calc

In [ ]:
# get two separate datasets based on location
data_a = calc[calc["is_dc_bin"] == 1]
data_b = calc[calc["is_dc_bin"] == 0]

data_a = data_a.rename(columns = {"c_current_year":"Current Year","c_next_year":"Next Year","c_second_year":"Year After Next"})
data_b = data_b.rename(columns = {"c_current_year":"Current Year","c_next_year":"Next Year","c_second_year":"Year After Next"})

In [ ]:
# data transformation in order to generate boxplot
data_a = pd.melt(data_a, id_vars=['c_row_id'], value_vars=['Current Year', 'Next Year', 'Year After Next'])
data_b = pd.melt(data_b, id_vars=['c_row_id'], value_vars=['Current Year', 'Next Year', 'Year After Next'])

In [ ]:
data_a

In [ ]:
# create boxplot for both regions of interest and over the three year period
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Box(
    x=data_a['value'],
    y=data_a['variable'],
    name='DC',
    marker_color='#701c62'
))
fig.add_trace(go.Box(
    x=data_b['value'],
    y=data_b['variable'],
    name='Non-DC',
    marker_color='#e75d64'
))


fig.update_layout(
    title='Hourly Rate of Washington, D.C. Region Contracts versus Rest of U.S.',
    xaxis_title='Hourly Rate ($)',
    boxmode='group', # group together boxes of the different traces for each value of x
    width=1000,
    height=500,
    template='plotly_white'
)
fig.update_traces(orientation='h')
fig.show()

In [ ]:
# BLS DATA

In [ ]:
# read in BLS data
bls = pd.read_excel('/Users/Jeanine/Downloads/tblFinal.xlsx')

In [ ]:
bls

In [ ]:
# count the occurrences of jobs within each state
state_bls_ct = Counter(bls['State_Code'])
dfstate_bls = pd.DataFrame.from_dict(state_bls_ct, orient='index').reset_index()

In [ ]:
# drop the row for no state and rename columns
dfstate_bls = dfstate_bls.drop(dfstate_bls.index[[4]])
dfstate_bls = dfstate_bls.rename(columns={"index": "fips", 0: "count"})

In [ ]:
dfstate_bls['count'] = pd.to_numeric(dfstate_bls['count'])

In [ ]:
# merge with dataset that knows the state codes
dfstate_bls = dfstate_bls.merge(statefips, how='left', on='fips')

In [ ]:
# plot state counts from CALC data
import plotly.express as px

fig = px.choropleth(dfstate_bls, locations="state", color="count", 
                    locationmode="USA-states",
                    scope="usa",
                    color_continuous_scale=px.colors.sequential.Sunsetdark,
                    range_color=(0,dfstate_bls['count'].max()),
                    title="National Compensation Survey Occupations by State")
fig.update_layout(coloraxis_colorbar=dict(
    title="Total Number of Jobs",
    thicknessmode="pixels", thickness=50,
    lenmode="pixels", len=350,
    yanchor="top", y=1,
    ticks="outside"
))
fig.show()

In [ ]:
# read in BLS data with the salary information
final = pd.read_csv('/Users/Jeanine/Downloads/BLS_Extract_2016_2020 (1).txt', 
                 sep = '|',encoding='unicode_escape', dtype='unicode',
                 index_col=False)

In [ ]:
# determine if data is in DC region
is_dc_bin = []

x = 0
while x < len(final):
    if final['area_title'][x] == 'Washington-Arlington-Alexandria, DC-VA-MD-WV Metropolitan Division':
        is_dc_bin.append(1)
    else:
        is_dc_bin.append(0)
    x += 1
    
final['is_dc_bin'] = is_dc_bin 

In [ ]:
# remove odd characters from salary and hourly columns
final = final[final['h_median'] != '*']
final = final[final['h_median'] != '#']
final['h_median'] = pd.to_numeric(final['h_median'])
final['a_median'] = final['a_median'].replace(',','', regex=True)
final['a_median'] = pd.to_numeric(final['a_median'])

In [ ]:
# split datasets based on DC area
final_a = final[final["is_dc_bin"] == 1]
final_b = final[final["is_dc_bin"] == 0]

In [ ]:
# create histogram of median salary distributions for the DC area as well as a line to indicate the "mean"
import statistics
fig = px.histogram(final_a, x="a_median", title='Distribution of Median Salaries in the Washington, D.C. Area',
                  template="simple_white", color_discrete_sequence = ['#701c62'],
                  labels = {'a_median':'Median Salary'})
fig.update_yaxes(title='Count')
fig.add_vline(x=statistics.mean(final_a['a_median']), line_width=3, line_dash="dash", line_color="#e75d64")
fig.show()

In [ ]:
# create histogram of median salary distributions for outside of DC area as well as a line to indicate the "mean"
fig = px.histogram(final_b, x="a_median", title='Distribution of Median Salaries Outside of the Washington, D.C. Area',
                  template="simple_white", color_discrete_sequence=['#e75d64'],
                  labels = {'a_median':'Median Salary'})
fig.update_yaxes(title='Count')
fig.add_vline(x=statistics.mean(final_b['a_median']), line_width=3, line_dash="dash", line_color="#701c62")
fig.show()